# Aufgabe 4: Evaluation

In [1]:
# Imports
import pandas as pd

## Import und Verarbeitung des Modells

In [2]:
from joblib import load
reg = load('gradientboosted.model')

In [3]:
dataset = pd.read_csv('./DatenAusgegeben1.0.csv', delimiter=';', encoding = 'cp852')

In [4]:
df_keinunnoetig = dataset.drop(['Verkaufsjahr', 'Verkaufsmonat', 'Erster Stock in qm', 'Zweiter Stock in qm', 'Pool', 'Küchen', 'Klimaanlage', 'Heizung', 'Preis'], axis = 1)
encode_kategorien = ['Bezirk', 'Lage', 'Heizungsqualitt', 'Typ', 'Steigung', 'Grundstücksform', 'Garage Typ', 'Zone']
encoded_kategorieeigenschaften = pd.get_dummies(dataset[encode_kategorien].fillna('Keine'))
df_encoded = df_keinunnoetig.drop(encode_kategorien, axis = 1).join(encoded_kategorieeigenschaften)

needed_columns = ['Grundstück in qm', 'Zustand', 'Gebaut', 'Renoviert', 'Zustand Fassade',
       'Kellerflche in qm', 'Wohnflche in qm', 'Schlafzimmer', 'Küchenqualitt',
       'Rume', 'Garagenkapazitt', 'Bezirk_Burnley', 'Bezirk_ChinaTown',
       'Bezirk_City Hall', 'Bezirk_Clear Creek', 'Bezirk_DiamondDistrict',
       'Bezirk_Dixon Side', 'Bezirk_East End', 'Bezirk_Fashion District',
       'Bezirk_Grand Park', 'Bezirk_Miller', 'Bezirk_New Gotham',
       'Bezirk_North East', 'Bezirk_North West', 'Bezirk_Novick District',
       'Bezirk_Old Gotham', 'Bezirk_Paris Island', 'Bezirk_Robinson Park',
       'Bezirk_Somerset', 'Bezirk_Somerset West', 'Bezirk_Somerst',
       'Bezirk_The Bowery', 'Bezirk_Tricorner', 'Bezirk_University',
       'Bezirk_Upper West', 'Bezirk_Water District', 'Lage_Bahn', 'Lage_Norm',
       'Lage_Pos', 'Lage_Str', 'Heizungsqualitt_Ex', 'Heizungsqualitt_Gut',
       'Heizungsqualitt_Schl', 'Heizungsqualitt_Ty', 'Heizungsqualitt_Ud',
       'Typ_1Fam', 'Typ_2Fam', 'Typ_2FamAn', 'Typ_REH', 'Typ_RMH',
       'Steigung_Ja', 'Steigung_Nein', 'Grundstücksform_IR1',
       'Grundstücksform_IR2', 'Grundstücksform_IR3', 'Grundstücksform_Reg',
       'Garage Typ_2Typen', 'Garage Typ_Anbau', 'Garage Typ_CarPort',
       'Garage Typ_Eingebaut', 'Garage Typ_Freistehend', 'Garage Typ_Keine',
       'Garage Typ_TG', 'Zone_RH', 'Zone_RL', 'Zone_RM',
       'Bezirk_Finger River']

print("Fehlende Eigenschaften:",set(needed_columns).difference(set(df_encoded.columns)))
print("Eigenschaften werden jetzt angefügt..")

for column in needed_columns:
    if column not in df_encoded:
        df_encoded[column] = 0

df_encoded = df_encoded.reindex(columns = needed_columns)

Fehlende Eigenschaften: {'Bezirk_Finger River'}
Eigenschaften werden jetzt angefügt..


In [5]:
y_pred = reg.predict(df_encoded)

In [6]:
dataset['Vorhergesagter Preis'] = y_pred

In [7]:
from sklearn.metrics import r2_score, mean_squared_error, max_error
import numpy as np

def mape (y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

## Metriken des finalen Modells

In [20]:
y_true = dataset['Preis']
print("R^2:", r2_score(y_true, y_pred))
print("MSE:", mean_squared_error(y_true, y_pred))
print("RMSE:", mean_squared_error(y_true, y_pred)**.5)
print("MAPE:", mape(y_true, y_pred))
print("MAX:", max_error(y_true, y_pred))

R^2: 0.8543983451026348
MSE: 755688028.388094
RMSE: 27489.78043542898
MAPE: 7.134673683837832
MAX: 426820.8922176695


## Evaluation der Vorhersage
Die Metriken zeigen uns, dass das in der Modeling Phase erstellte Modell gut geeignet ist, eine Preisvorhersage wie in der Phase des Business Understandings definiert durchzuführen.  

Hohe Werte des MAX-Error und des MSE/RMSE sind auf Ausreißer im durchlaufenen Datensatz zurückzuführen.  



## Evaluation der anderen Ziele
In der Data Exploration und Analyse wurde versucht die anderen Ziele, die Frage nach Renovierungsmöglichkeiten,  die Frage mit welchem Budget in welchen Bezirk investiert werden sollte, die Frage welche Hausattribute einen ausschlaggebenden Einfluss auf den Preis besitzen und die Frage nach dem profitabelsten Verkaufsmonat einer Immobilie zu beantworten.

Es konnten alle Fragen, außer der nach dem profitabelsten Verkaufsmonat in der Data Exploration und Analyse beantwortet werden. Zu dieser Frage konnte keine Antwort gefunden werden, da es sich herausstellte, dass der Verkaufsmonat in diesem Datensatz keinen Einfluss auf den Verkaufspreis einer Immobilie besitzt.